open "C:/Users/LoydY/OneDrive - SUNY Upstate Medical University/Documents/20240917Phagocytosis/20240917_RAWN164A_002_actin.tif";

open "C:/Users/LoydY/OneDrive - SUNY Upstate Medical University/Documents/20240917Phagocytosis/20240917_RAWN164A_002_m1e.tif";

open "C:/Users/LoydY/OneDrive - SUNY Upstate Medical University/Documents/20240917Phagocytosis/20240917_RAWN164A_002_beads.tif";

volume #1 style surface voxelsize 1,1,1.5;
marker #2 position 522,815,35.35 color yellow radius 10;

ui tool show shell

In [ ]:
from chimerax.map_data import ArrayGridData 
from chimerax.map import volume_from_grid_data
import itertools
import numpy
import scipy
import skimage

In [ ]:
y=session.models[2].volume.full_matrix().copy()
p= session.models[3].atoms.coords
p=numpy.round(p).astype('int')

matrix=numpy.zeros((numpy.shape(y)))
a=numpy.linspace(p[0,0]-80,p[0,0]+80, 161)
b=numpy.linspace(p[0,1]-80,p[0,1]+80, 161)
c=numpy.linspace((p[0,2]/1.5)-15,(p[0,2]/1.5)+15, 31)

q=numpy.array(list(itertools.product(*(a,b,c),repeat=1)),dtype='int')
w=numpy.split(q,3,1)
matrix[w[2],w[1],w[0]]=1
mask=ArrayGridData(matrix,step=(1,1,1.5))
volume_from_grid_data(mask,session)

This draws a bos around a marker. Coordnet set is derived from a marker set. 

In [ ]:
y=session.models[2].volume.full_matrix().copy()
p= session.models[3].atoms.coords
p=numpy.round(p).astype('int')

for t in range (numpy.shape(p)[0]):
    matrix=numpy.zeros((numpy.shape(y)))
    a=numpy.linspace(p[t,0]-80,p[t,0]+80, 161)
    b=numpy.linspace(p[t,1]-80,p[t,1]+80, 161)
    c=numpy.round(numpy.linspace(p[t,2]-15,p[t,2]+15, 31) / 2.5)

    q=numpy.array(list(itertools.product(*(a,b,c),repeat=1)),dtype='int')
    w=numpy.split(q,3,1)
    matrix[w[2],w[1],w[0]]=1
    mask=ArrayGridData(matrix,step=(1,1,2.5))
    volume_from_grid_data(mask,session)

In [ ]:
'Attempt to add a roof and floor'

for t in range (numpy.shape(p)[0]):
    matrix=numpy.zeros((numpy.shape(y)))
    a=numpy.linspace(p[t,0]-80,p[t,0]+80, 161)
    b=numpy.linspace(p[t,1]-80,p[t,1]+80, 161)
    zip= numpy.round(p[t,2]/2.5).astype('int')
    c=numpy.linspace((zip-15),(zip+15), 31)
    """Shift down one accounting for 0 start"""
    a=a-1
    b=b-1
    c=c-1

    """Gates"""
    j=numpy.delete(a,numpy.where(a>=numpy.shape(matrix)[2]),axis=0) 
    j=numpy.delete(j,numpy.where(j<0),axis=0)

    k=numpy.delete(b,numpy.where(b>=numpy.shape(matrix)[1]),axis=0) 
    k=numpy.delete(k,numpy.where(k<0),axis=0)

    l=numpy.delete(c,numpy.where(c>=numpy.shape(matrix)[0]),axis=0) 
    l=numpy.delete(l,numpy.where(l<0),axis=0)

    q=numpy.array(list(itertools.product(*(j,k,l),repeat=1)),dtype='int')
    w=(numpy.split(q,3,1))
    matrix[w[2],w[1],w[0]]=1
    mask=ArrayGridData(matrix,step=(1,1,2.5))
    volume_from_grid_data(mask,session)

Draws boxs to a limited ROI failed on 1/22 marks.

Will need to test limitiations and move onto auth Threshold concept

In [ ]:
y=session.models[2].volume.full_matrix().copy()
p= session.models[3].atoms.coords
p=numpy.round(p).astype('int')

for t in range (numpy.shape(p)[0]):
    matrix=numpy.zeros((numpy.shape(y)))
    a=numpy.linspace(p[t,0]-125,p[t,0]+125, 251)
    b=numpy.linspace(p[t,1]-125,p[t,1]+125, 251)
    zip= numpy.round(p[t,2]/1.5).astype('int')
    c=numpy.linspace((zip-40),(zip+40), 81)
    """Shift down one accounting for 0 start"""
    a=a-1
    b=b-1
    c=c-1

    """Gates"""
    j=numpy.delete(a,numpy.where(a>=numpy.shape(matrix)[2]),axis=0) 
    j=numpy.delete(j,numpy.where(j<0),axis=0)

    k=numpy.delete(b,numpy.where(b>=numpy.shape(matrix)[1]),axis=0) 
    k=numpy.delete(k,numpy.where(k<0),axis=0)

    l=numpy.delete(c,numpy.where(c>=numpy.shape(matrix)[0]),axis=0) 
    l=numpy.delete(l,numpy.where(l<0),axis=0)

    q=numpy.array(list(itertools.product(*(j,k,l),repeat=1)),dtype='int')
    w=(numpy.split(q,3,1))
    matrix[w[2],w[1],w[0]]=1
    mask=ArrayGridData(matrix,step=(1,1,1.5))
    volume_from_grid_data(mask,session)

In [ ]:
welp=matrix*y
mask=ArrayGridData(welp,step=(1,1,1.5))
volume_from_grid_data(mask,session)


bernson auto-thresh (failed as capallaries are in close proximity to eachother in 3d might consider a 2d iterative approach )

In [ ]:
welp=matrix*y
welp[welp==0] = numpy.nan
radius=10
help= numpy.zeros(numpy.shape(welp))
for h in range(numpy.min(w[2]),numpy.max(w[2])):
    for g in range(numpy.min(w[1]),numpy.max(w[1])):
        for f in range(numpy.min(w[0]),numpy.max(w[0])):
            fl=numpy.int32(numpy.linspace(f-radius,radius+f, 2*radius+1))-1
            gl=numpy.int32(numpy.linspace(g-radius,radius+g, 2*radius+1))-1
            hl=numpy.int32(numpy.linspace(h-radius,radius+h, 2*radius+1))-1
            Thresh= (numpy.nanmax(welp[hl,gl,fl])-numpy.nanmin(welp[hl,gl,fl]))/2
            contrast= (numpy.nanmax(welp[hl,gl,fl])-numpy.nanmin(welp[hl,gl,fl]))
            if contrast < 450:
                welp[h,g,f]= numpy.nan
            else:
                welp[h,g,f]= welp[h,g,f]
            
            if welp[h,g,f] >= (Thresh):
                help[h,g,f]= 1
            else:
                help[h,g,f]=0

In [ ]:
welp=matrix*y
welp[welp==0] = numpy.nan
radius=5
help= numpy.zeros(numpy.shape(welp))
for h in range(numpy.min(w[2]),numpy.max(w[2])):
    for g in range(numpy.min(w[1]),numpy.max(w[1])):
        for f in range(numpy.min(w[0]),numpy.max(w[0])):
            fl=numpy.int32(numpy.linspace(f-radius,radius+f, 2*radius+1))-1
            gl=numpy.int32(numpy.linspace(g-radius,radius+g, 2*radius+1))-1
            hl=numpy.int32(numpy.linspace(h-radius,radius+h, 2*radius+1))-1
            standardD = numpy.nanstd(welp[hl,gl,fl])*0.15
            Thresh= ((numpy.nanmax(welp[hl,gl,fl])-numpy.nanmin(welp[hl,gl,fl]))/2)-standardD
            contrast= (numpy.nanmax(welp[hl,gl,fl])-numpy.nanmin(welp[hl,gl,fl]))-standardD
            
            if contrast < 450:
                welp[h,g,f]= numpy.nan
            else:
                welp[h,g,f]= welp[h,g,f]
            
            if welp[h,g,f] >= (Thresh):
                help[h,g,f]= 1
            else:
                help[h,g,f]=0

simple mean at

In [ ]:
welp[welp==0] = numpy.nan 
radius=10
help= numpy.zeros(numpy.shape(welp))
for h in range(numpy.min(w[2]),numpy.max(w[2])):
    for g in range(numpy.min(w[1]),numpy.max(w[1])):
        for f in range(numpy.min(w[0]),numpy.max(w[0])):
            fl=numpy.int32(numpy.linspace(f-radius,radius+f, 2*radius+1))-1
            gl=numpy.int32(numpy.linspace(g-radius,radius+g, 2*radius+1))-1
            hl=numpy.int32(numpy.linspace(h-radius,radius+h, 2*radius+1))-1
            local_avg= numpy.nanmean(welp[hl,gl,fl])
            if welp[h,g,f] >= (local_avg):
                help[h,g,f]= 1
            else:
                help[h,g,f]=0
            

In [ ]:
welp=matrix*y
welp[welp==0] = numpy.nan
radius=10
help= numpy.zeros(numpy.shape(welp))
for h in range(numpy.nanmin(w[2]),numpy.nanmax(w[2])):
    for g in range(numpy.nanmin(w[1]),numpy.nanmax(w[1])):
        for f in range(numpy.nanmin(w[0]),numpy.nanmax(w[0])):
            fl=numpy.int32(numpy.linspace(f-radius,radius+f, 2*radius+1))-1
            gl=numpy.int32(numpy.linspace(g-radius,radius+g, 2*radius+1))-1
            hl=numpy.int32(numpy.linspace(h-radius,radius+h, 2*radius+1))-1
            Thresh= (numpy.nanmax(welp[hl,gl,fl])-numpy.nanmin(welp[hl,gl,fl]))/2
            contrast= (numpy.nanmax(welp[hl,gl,fl])-numpy.nanmin(welp[hl,gl,fl]))
            if contrast < 450:
                welp[h,g,f]= numpy.nan
            else:
                welp[h,g,f]= welp[h,g,f]
            
            if welp[h,g,f] >= (Thresh):
                help[h,g,f]= 1
            else:
                help[h,g,f]=0

In [ ]:
welp=matrix*y
welp[welp==0] = numpy.nan 
radius=10
elp= numpy.zeros(numpy.shape(welp))
for h in range(numpy.nanmin(w[2]),numpy.nanmax(w[2])):
    for g in range(numpy.nanmin(w[1]),numpy.nanmax(w[1])):
        for f in range(numpy.nanmin(w[0]),numpy.nanmax(w[0])):
            fl=numpy.int32(numpy.linspace(f-radius,radius+f, 2*radius+1))-1
            gl=numpy.int32(numpy.linspace(g-radius,radius+g, 2*radius+1))-1
            hl=numpy.int32(numpy.linspace(h-radius,radius+h, 2*radius+1))-1
            local_avg= numpy.nanmean(welp[hl,gl,fl])
            if welp[h,g,f] >= (local_avg):
                elp[h,g,f]= 1
            else:
                elp[h,g,f]=0

overlap of mask

In [ ]:
yelp = help*elp
mask=ArrayGridData(yelp,step=(1,1,1.5))
volume_from_grid_data(mask,session)

fail much close to the ideal threshold

In [ ]:
welp=matrix*y
welp[welp==0] = numpy.nan 
radius=6
elp= numpy.zeros(numpy.shape(welp))
for h in range(numpy.nanmin(w[2]),numpy.nanmax(w[2])):
    for g in range(numpy.nanmin(w[1]),numpy.nanmax(w[1])):
        for f in range(numpy.nanmin(w[0]),numpy.nanmax(w[0])):
            fl=numpy.int32(numpy.linspace(f-radius,radius+f, 2*radius+1))-1
            gl=numpy.int32(numpy.linspace(g-radius,radius+g, 2*radius+1))-1
            hl=numpy.int32(numpy.linspace(h-radius,radius+h, 2*radius+1))-1
            local_avg= numpy.nanmean(welp[hl,gl,fl])
            stan=numpy.nanstd(welp[hl,gl,fl])
            if welp[h,g,f] >= (local_avg-(0.5*stan)):
                elp[h,g,f]= 1
            else:
                elp[h,g,f]=0

Sauvola thresholding

In [ ]:
welp=matrix*y
welp[welp==0] = numpy.nan 
radius=5
elp= numpy.zeros(numpy.shape(welp))
for h in range(numpy.nanmin(w[2]),numpy.nanmax(w[2])):
    for g in range(numpy.nanmin(w[1]),numpy.nanmax(w[1])):
        for f in range(numpy.nanmin(w[0]),numpy.nanmax(w[0])):
            fl=numpy.int32(numpy.linspace(f-radius,radius+f, 2*radius+1))-1
            gl=numpy.int32(numpy.linspace(g-radius,radius+g, 2*radius+1))-1
            hl=numpy.int32(numpy.linspace(h-radius,radius+h, 2*radius+1))-1
            local_avg= numpy.nanmean(welp[hl,gl,fl])
            stan=numpy.nanstd(welp[hl,gl,fl])
            R=numpy.nanmax(welp[hl,gl,fl])-numpy.nanmin(welp[hl,gl,fl])
            if welp[h,g,f] >= (local_avg-(0.5*((stan/R)-1))):
                elp[h,g,f]= 1
            else:
                elp[h,g,f]=0

sauvola - roi wedge (reduced z radius)

In [ ]:
welp=matrix*y
welp[welp==0] = numpy.nan 
radius=4
zradius=2
elp= numpy.zeros(numpy.shape(welp))
for h in range(numpy.nanmin(w[2]),numpy.nanmax(w[2])):
    for g in range(numpy.nanmin(w[1]),numpy.nanmax(w[1])):
        for f in range(numpy.nanmin(w[0]),numpy.nanmax(w[0])):
            fl=numpy.int32(numpy.linspace(f-radius,radius+f, 2*radius+1))-1
            gl=numpy.int32(numpy.linspace(g-radius,radius+g, 2*radius+1))-1
            hl=numpy.int32(numpy.linspace(h-zradius,zradius+h, 2*zradius+1))-1
            
            ql=numpy.array(list(itertools.product(*(fl,gl,hl),repeat=1)),dtype='int')
            wl=(numpy.split(ql,3,1))
            
            local_avg= numpy.nanmean(welp[wl[2],wl[1],wl[0]])
            stan=numpy.nanstd(welp[wl[2],wl[1],wl[0]])
            R=numpy.nanmax(welp[wl[2],wl[1],wl[0]])-numpy.nanmin(welp[wl[2],wl[1],wl[0]])
            if welp[h,g,f] >= (local_avg-(0.5*((stan/R)-1))):
                elp[h,g,f]= 1
            else:
                elp[h,g,f]=0

sauvola - roi 2d square iterative - same as wedge

In [ ]:
welp=matrix*y
welp[welp==0] = numpy.nan 
radius=5
zradius=0
elp= numpy.zeros(numpy.shape(welp))
for h in range(numpy.nanmin(w[2]),numpy.nanmax(w[2])):
    for g in range(numpy.nanmin(w[1]),numpy.nanmax(w[1])):
        for f in range(numpy.nanmin(w[0]),numpy.nanmax(w[0])):
            fl=numpy.int32(numpy.linspace(f-radius,radius+f, 2*radius+1))-1
            gl=numpy.int32(numpy.linspace(g-radius,radius+g, 2*radius+1))-1
            hl=numpy.int32(numpy.linspace(h-zradius,zradius+h, 2*zradius+1))-1
            
            ql=numpy.array(list(itertools.product(*(fl,gl,hl),repeat=1)),dtype='int')
            wl=(numpy.split(ql,3,1))
            
            local_avg= numpy.nanmean(welp[wl[2],wl[1],wl[0]])
            stan=numpy.nanstd(welp[wl[2],wl[1],wl[0]])
            R=numpy.nanmax(welp[wl[2],wl[1],wl[0]])-numpy.nanmin(welp[wl[2],wl[1],wl[0]])
            if welp[h,g,f] >= (local_avg-(0.5*((stan/R)-1))):
                elp[h,g,f]= 1
            else:
                elp[h,g,f]=0

In [ ]:
y=session.models[2].volume.full_matrix().copy()
p= session.models[3].atoms.coords
p=numpy.round(p).astype('int')

matrix=numpy.zeros((numpy.shape(y)))
a=numpy.linspace(p[0,0]-200,p[0,0]+200, 401)
b=numpy.linspace(p[0,1]-200,p[0,1]+200, 401)
c=numpy.linspace((p[0,2]/1.5)-20,(p[0,2]/1.5)+20, 41)

q=numpy.array(list(itertools.product(*(a,b,c),repeat=1)),dtype='int')
w=numpy.split(q,3,1)
matrix[w[2],w[1],w[0]]=1

welp=matrix*y
welp[welp==0] = numpy.nan
radius=3
help= numpy.zeros(numpy.shape(welp))
for h in range(numpy.min(w[2]),numpy.max(w[2])):
    for g in range(numpy.min(w[1]),numpy.max(w[1])):
        for f in range(numpy.min(w[0]),numpy.max(w[0])):
            fl=numpy.int32(numpy.linspace(f-radius,radius+f, 2*radius+1))-1
            gl=numpy.int32(numpy.linspace(g-radius,radius+g, 2*radius+1))-1
            hl=numpy.int32(numpy.linspace(h-radius,radius+h, 2*radius+1))-1
            if numpy.isnan((welp[hl,gl,fl].all()))==False:
                standardD = numpy.nanstd(welp[hl,gl,fl])*0.15
                Thresh= ((numpy.nanmax(welp[hl,gl,fl])-numpy.nanmin(welp[hl,gl,fl]))/2)+standardD
                contrast= (numpy.nanmax(welp[hl,gl,fl])-numpy.nanmin(welp[hl,gl,fl]))
            else:
                help = numpy.nan
            
            if contrast < 450:
                welp[h,g,f]= numpy.nan
            else:
                welp[h,g,f]= welp[h,g,f]
            
            if welp[h,g,f] >= (Thresh):
                help[h,g,f]= 1
            else:
                help[h,g,f]=0

In [ ]:

telp=skimage.morphology.erosion(help)

telp=skimage.morphology.dilation(telp)